<a href="https://colab.research.google.com/github/ipeirotis/dealing_with_data/blob/master/06-Spatial_Data_and_Maps/D-Citibike_Station_Visualization_using_Folium.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Visualization of Citibike Stations using Folium


Another perfect tool for vizualisation data processed in Python is `folium`. It builds on the data wrangling strengths of the Python ecosystem and the mapping strengths of the Leaflet.js library. It manipulate your data in Python, then visualize it in on a Leaflet map.

In [ ]:
# @title Setup

!pip install -U -q geopandas tdqm folium mapclassify folium

import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import folium
import requests


%config InlineBackend.figure_format = 'retina'

# Change the graph defaults
plt.rcParams['figure.figsize'] = (8, 3)  # Default figure size of 6x2 inches
plt.rcParams['axes.grid'] = True
plt.rcParams['grid.color'] = 'lightgray'
plt.rcParams['font.size'] = 10  # Default font size of 12 points
plt.rcParams['lines.linewidth'] = 1  # Default line width of 1 points
plt.rcParams['lines.markersize'] = 3  # Default marker size of 3 points
plt.rcParams['legend.fontsize'] = 10  # Default legend font size of 10 poin

## Citibike API

Let's get the data from the Citibike API.

In [ ]:
# Fetch the station info
url1 = 'https://gbfs.lyft.com/gbfs/2.3/bkn/en/station_information.json'
results1 = requests.get(url1).json()
data1 = results1["data"]["stations"]
citibike1 = pd.DataFrame(data1)

# And fetch the number of bikes currently in each station
url2 = "https://gbfs.lyft.com/gbfs/2.3/bkn/en/station_status.json"
results2 = requests.get(url2).json()
data2 = results2["data"]["stations"]

citibike2 = pd.DataFrame(data2)

citibike = pd.merge(citibike1, citibike2, on='station_id')

# Keep only variables of interest
citibike = citibike.filter( ['station_id',  'name',  'lon', 'lat',
       'capacity', 'is_installed', 'is_renting', 'is_returning',
       'num_bikes_available',
       'num_docks_available',
       'num_ebikes_available',
        'num_bikes_disabled',
       'num_docks_disabled', 'num_scooters_available',
       'num_scooters_unavailable'] )

In [ ]:
citibike

## Selecting tiles

The first step is to create a map. At the very basic level, we select the location, zoom level, and potentially the tiles (i.e., the style of the map) for the background. The default is 'OpenStreetMap', but often for visualizations we prefer other, more visually neutral styles. (See http://folium.readthedocs.io/en/latest/quickstart.html for more tile optionS)

In [ ]:
fmap = folium.Map(location=[40.85, -73.8], zoom_start=12, tiles='OpenStreetMap')
fmap

In [ ]:
fmap = folium.Map(location=[40.85, -73.8], zoom_start=12, tiles='cartodbpositron')
fmap

## Adding Markers

For every station, we are going to add a marker in the map:
* Using the longitude and latitude for the location
* Modify the color of the marker to reflect the status of the station
* Modify the opacity to be the percentage of bikes in the station.
* Modify the size of the circle to corresponds to the size of the station.

In [ ]:
fmap = folium.Map(location=[citibike.lat.mean(), citibike.lon.mean()], zoom_start=12,  tiles='cartodbpositron')

for name, row in citibike.iterrows():

    # Define the opacity of the marker to be proportional to the percentage of bikes in the station
    opacity = row["num_bikes_available"]/row["capacity"] if row["capacity"] > 0 else 1.0
    # Make the color green for the working stations, red otherwise
    color = "green" if row["is_installed"] == 1 else "red"
    # The size of the marker is proportional to the number of docks
    size = row["capacity"]/10 if row["is_installed"] == 1 else 3

    # We create a marker on the map and we add it to the map
    folium.CircleMarker(location=[row["lat"], row["lon"]],
                        radius = size,
                        color='black', weight=0.5,
                        fill=True,
                        fill_opacity = opacity,
                        fill_color = color,
                       ).add_to(fmap)

fmap

## Adding popups to the markers

For each marker, we can also have a popup with text, html, or even other charts/visualizations. Here is an example of adding an HTML popup to each marker.

In [ ]:
fmap = folium.Map(location=[citibike.lat.mean(), citibike.lon.mean()], zoom_start=12,  tiles='cartodbpositron')

for name, row in citibike.iterrows():

    # Define the opacity of the marker to be proportional to the percentage of bikes in the station
    opacity = row["num_bikes_available"]/row["capacity"] if row["capacity"] > 0 else 1.0
    # Make the color green for the working stations, red otherwise
    color = "green" if row["is_installed"] == 1 else "red"
    # The size of the marker is proportional to the number of docks
    size = row["capacity"]/10 if row["is_installed"] == 1 else 3

    # The code below defines a pop-up for each station with details such as
    # the address, number of bikes, capacity, etc.
    html = f"""
            <p style='font-family:sans-serif;font-size:11px'>
            <strong>Address:</strong> {row["name"]} <br>
            <strong>Available Bikes:</strong>  {row["num_bikes_available"]}<br>
            <strong>Total Docks: </strong> {row["capacity"]}
            """
    iframe = folium.IFrame(html=html, width=200, height=60)
    popup = folium.Popup(iframe, max_width=200)

    # We create a marker on the map and we add it to the map
    folium.CircleMarker(location=[row["lat"], row["lon"]],
                        radius = size,
                        popup = popup,
                        color='black', weight=0.5,
                        fill=True,
                        fill_opacity = opacity,
                        fill_color = color,
                       ).add_to(fmap)

fmap

## (Optional) Saving the Folium map in a static file

In [ ]:
# @title Save the map as an HTML file
filepath = 'citibike.html'
fmap.save(filepath)

In [ ]:
# @title Save the map as an PNG file

!apt-get update > /dev/null # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver > /dev/null
!pip install -U -q selenium

import os
from selenium import webdriver
import time

# Save the map as an HTML file
filepath = 'citibike.html'
fmap.save(filepath)

# Set up the Selenium webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome(options=chrome_options)

# Load the saved map with Selenium
file_url='file://{path}/{mapfile}'.format(path=os.getcwd(),mapfile=filepath)
wd.get(file_url)
time.sleep(5)  # Wait a few seconds for the map to fully load

# Take a screenshot of the map
wd.save_screenshot('citibike.png')

# Close the Selenium webdriver
wd.quit()
